In [1]:
import math
import numpy as np
import os
import pickle
import rasterio
import xarray as xr
from rasterio.transform import from_origin
from pathlib import Path
from metpy.calc import showalter_index
from metpy.units import units
from metpy.interpolate import interpolate_1d
from metpy.calc import dewpoint_from_relative_humidity

import warnings
warnings.filterwarnings("ignore")

In [2]:
# '''
# Lifted condensation level: The height at which air cools enough to condense water vapor into liquid (like clouds forming).
# If the air temperature is much higher than the dew point, this level will be lower.

# If LCL is above 500 hPa: 
#     The air hasn't reached its dew point yet, so it uses a "dry" cooling rate to calculate how much cooler the air would be at 500 hPa.
    
# If LCL is below 500 hPa:
#     The air has reached its dew point, so it uses a "moist" cooling rate instead, which accounts for the fact that moisture in the air affects how it cools.
# '''

In [3]:
def calculate_dew_point(relative_humidity, air_temp):
    air_temp = air_temp - 273.15 # in degC
    
    dew_point = (air_temp - (100 - relative_humidity) / 5)
    return dew_point # in degC

# air parcel is lifted dry-adiabatically from 850 hPa level to the lifted condensation level (LCL) and from LCL up to 500 hPa moist-adiabatically.
def calculate_Tp500(relative_humidity_850, T_850):
    pressure_levels = np.arange(850, 475, -25) * 100 # in Pa

    initial_temp = T_850 - 273.15 # in degC
    initial_dewpoint = calculate_dew_point(relative_humidity_850, T_850) #in degC

    dry_lapse_rate = 9.8 / 1000   # Convert to degC/m
    moist_lapse_rate = 4.8 / 1000  # Convert to degC/m   
    initial_pressure = 850 * 100 # in Pa

    parcel_profile = np.zeros((len(pressure_levels),) + T_850.shape)

    parcel_profile[0] = initial_temp

    for i in range(1, len(pressure_levels)):
        height_diff = (
            (pressure_levels[i - 1] - pressure_levels[i]) 
            / 101325 
            * 287.05 
            * (parcel_profile[i - 1] + 273.15) 
            / pressure_levels[i - 1]
        )  # Approximate height difference

        is_dry = parcel_profile[i - 1] > initial_dewpoint
        lapse_rate = np.where(is_dry, dry_lapse_rate, moist_lapse_rate)

        parcel_profile[i] = parcel_profile[i - 1] - lapse_rate * height_diff

    return parcel_profile[-1] # in degC

In [4]:
folder = Path('GFS_LFTX')

lftx_list = []

# Get all file paths in the folder
lftx_files = [file for file in folder.iterdir() if file.is_file() and file.name.startswith('lftx')]
lftx_files.sort()

for lftx_file in lftx_files:

    lftx = xr.open_dataset(lftx_file)
    lftx = lftx['LFT_X_L1']
    lftx_list.append(lftx)
    
print(len(lftx_list))

21


In [ ]:
folder = Path('GFS_RHT')

# Get all file paths in the folder
gfs_files = [file for file in folder.iterdir() if file.is_file() and file.name.startswith('gfs')]
print(len(gfs_files))
gfs_files.sort()

for i in range(len(gfs_files)):
    print(f"Opened: {gfs_files[i]}")

    gfs_data = xr.open_dataset(gfs_files[i])

    levels = [850, 700, 500]  # Isobaric levels in hPa

    # Extract latitude and longitude
    lats = gfs_data['lat'].values
    lons = gfs_data['lon'].values
    lon_grid, lat_grid = np.meshgrid(lons, lats)

    # Define the transform (top-left corner coordinates and resolution)
    transform = from_origin(lon_grid.min(), lat_grid.max(), np.abs(lons[1] - lons[0]), np.abs(lats[1] - lats[0]))

    # Coordinate reference system (WGS 84)
    crs = 'EPSG:4326'

    # Prepare an empty list to store the data arrays for each variable
    bands_data = []

    T_850 = gfs_data['TMP_L100'].sel(level0=850)
    T_700 = gfs_data['TMP_L100'].sel(level0=700)
    T_500 = gfs_data['TMP_L100'].sel(level0=500)

    RH_850 = gfs_data['R_H_L100'].sel(level0=850)
    RH_700 = gfs_data['R_H_L100'].sel(level0=700)
    
    Td_700 = calculate_dew_point(RH_700, T_700) # in degC
    Td_850 = calculate_dew_point(RH_850, T_850) # in degC

    Tp_500 = calculate_Tp500(RH_850, T_850) # in degC

    # K = (T850 -T500) + Td850 - (T700-Td700)
    k_index = (T_850 - T_500) + Td_850 - ((T_700 - 273.15) - Td_700)
    # shw_index = (T_500.values - 273.15) - Tp_500
    lifted_index = lftx_list[i]

    # print("stable", (k_index < 20).sum(), (shw_index > 3).sum(), (lifted_index > 0).sum())
    print("stable", (k_index < 20).sum(), (lifted_index > 0).sum())

    # print(k_index.shape, shw_index.shape, lifted_index.shape)
    print("KI", k_index.values.min(), k_index.values.max())
    # print("SI", shw_index.min(), shw_index.max())
    print("LI", lifted_index.values.min(), lifted_index.values.max())
    
    bands_data.append(k_index.values)
    # bands_data.append(shw_index)
    bands_data.append(lifted_index.values)
    # print(type(bands_data[0]), type(bands_data[1]), type(bands_data[2]))
    # print(bands_data[0].shape, bands_data[1].shape, bands_data[2].shape)

    # Convert list to a numpy array for multiband GeoTIFF
    bands_data = np.stack(bands_data, axis=0)

    bands_data = np.squeeze(bands_data) # (2, 721, 1440)

    filename = os.path.basename(gfs_files[i])
    parts = filename.split('.grib2.nc')

    output_tiff_path = f'Geotiff_GFS/{parts[0]}.tif'

    # Create a new GeoTIFF file with the multiband data
    with rasterio.open(
        output_tiff_path,
        'w',
        driver='GTiff',
        height=bands_data.shape[1],  # The height (lat dimension)
        width=bands_data.shape[2],   # The width (lon dimension)
        count=bands_data.shape[0],   # The number of bands
        dtype=bands_data.dtype,
        crs=crs,
        transform=transform,
    ) as dst:
        for i in range(bands_data.shape[0]):
            dst.write(bands_data[i, :, :], i + 1) # Write each band

    print(f"GeoTIFF created at: {output_tiff_path}")

21
Opened: GFS_RHT/gfs.0p25.2024070918.f000.grib2.nc
stable <xarray.DataArray ()> Size: 8B
array(628271) <xarray.DataArray 'LFT_X_L1' ()> Size: 8B
array(803204)
KI -84.68741 54.982613
LI -14.319756 49.580246
GeoTIFF created at: Geotiff_GFS/gfs.0p25.2024070918.f000.tif
Opened: GFS_RHT/gfs.0p25.2024071000.f000.grib2.nc
stable <xarray.DataArray ()> Size: 8B
array(638956) <xarray.DataArray 'LFT_X_L1' ()> Size: 8B
array(810232)
KI -82.89215 55.05783
LI -14.479846 49.220154
GeoTIFF created at: Geotiff_GFS/gfs.0p25.2024071000.f000.tif
Opened: GFS_RHT/gfs.0p25.2024071006.f000.grib2.nc
stable <xarray.DataArray ()> Size: 8B
array(639989) <xarray.DataArray 'LFT_X_L1' ()> Size: 8B
array(800604)
KI -76.03813 58.251854
LI -13.596586 49.803413
GeoTIFF created at: Geotiff_GFS/gfs.0p25.2024071006.f000.tif
Opened: GFS_RHT/gfs.0p25.2024071012.f000.grib2.nc
stable <xarray.DataArray ()> Size: 8B
array(639523) <xarray.DataArray 'LFT_X_L1' ()> Size: 8B
array(795054)
KI -64.0196 56.93036
LI -12.713897 50.1861

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pyproj import CRS
import matplotlib.pyplot as plt

folder = Path('Geotiff_GFS')

# Get all file paths in the folder
gfs_files = [file for file in folder.iterdir() if file.is_file() and file.name.endswith('.tif')]
print(len(gfs_files))
gfs_files.sort()
# Define custom Mercator projection using PROJ string
mercator_crs = CRS.from_string(
    "+proj=merc +a=6378137 +b=6378137 +lat_ts=0 +lon_0=75 +x_0=0 +y_0=0 +k=1 +units=m +nadgrids=@null +wktext +no_defs"
)

for gfs_filepath in gfs_files:
    print(f"Opened: {gfs_filepath}")

    filename = os.path.basename(gfs_filepath)
    output_file = f'Reprojected_GFS/{filename}'

    with rasterio.open(gfs_filepath) as src:
        
        # Read metadata and data
        data = src.read()
        # print("rawwwr", (data[0] < 20).sum(), (data[1] > 3).sum(), (data[2] > 0).sum())
        print("rawwwr", (data[0] < 20).sum(), (data[1] > 0).sum())
        profile = src.profile

        # Define the destination CRS
        dst_crs = mercator_crs.to_proj4()

        # Calculate the transform and size for the destination CRS
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )

        # Update the profile for the output file
        profile.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height,
            'dtype': 'float32'
        })

        # Reproject the data and write it to the new file
        with rasterio.open(output_file, 'w', **profile) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )

        print(f"Reprojection complete. Output saved to {output_file}")

21
Opened: Geotiff_GFS/gfs.0p25.2024070918.f000.tif
rawwwr 628271 803204
Reprojection complete. Output saved to Reprojected_GFS/gfs.0p25.2024070918.f000.tif
Opened: Geotiff_GFS/gfs.0p25.2024071000.f000.tif
rawwwr 638956 810232
Reprojection complete. Output saved to Reprojected_GFS/gfs.0p25.2024071000.f000.tif
Opened: Geotiff_GFS/gfs.0p25.2024071006.f000.tif
rawwwr 639989 800604
Reprojection complete. Output saved to Reprojected_GFS/gfs.0p25.2024071006.f000.tif
Opened: Geotiff_GFS/gfs.0p25.2024071012.f000.tif
rawwwr 639523 795054
Reprojection complete. Output saved to Reprojected_GFS/gfs.0p25.2024071012.f000.tif
Opened: Geotiff_GFS/gfs.0p25.2024071018.f000.tif
rawwwr 641491 808422
Reprojection complete. Output saved to Reprojected_GFS/gfs.0p25.2024071018.f000.tif
Opened: Geotiff_GFS/gfs.0p25.2024071100.f000.tif
rawwwr 648044 812478
Reprojection complete. Output saved to Reprojected_GFS/gfs.0p25.2024071100.f000.tif
Opened: Geotiff_GFS/gfs.0p25.2024071106.f000.tif
rawwwr 650143 804612
Rep

In [ ]:
import numpy as np
import rasterio
from scipy.interpolate import griddata
from collections import Counter
from pathlib import Path

interpolated_gfs_data_list = []

# Function to compute the coordinates from the affine transform
def compute_coordinates(transform, width, height):
    x_coords, y_coords = np.meshgrid(
        np.arange(width) * transform[0] + transform[2],
        np.arange(height) * transform[4] + transform[5]
    )
    return x_coords, y_coords

# Path to the folder containing the GFS files
folder = Path('Reprojected_GFS')

# Get all file paths in the folder
gfs_files = [file for file in folder.iterdir() if file.is_file() and file.name.startswith('gfs')]
print(len(gfs_files))
gfs_files.sort()

# Load and compute the coordinates for the INSAT dataset (common to all GFS files)
insat_filepath = 'INSAT/3RIMG_10JUL2024_0015_L1C_SGP_V01R00_IMG_TIR1.tif'
with rasterio.open(insat_filepath) as insat_src:
    insat_transform = insat_src.transform
    insat_width = insat_src.width
    insat_height = insat_src.height
    insat_x, insat_y = compute_coordinates(insat_transform, insat_width, insat_height)

# Prepare the INSAT grid where GFS data will be interpolated
insat_points = np.column_stack((insat_x.flatten(), insat_y.flatten()))

# Process each GFS file
for idx, gfs_filepath in enumerate(gfs_files):
    print(f"\nOpened: {gfs_filepath}")
    with rasterio.open(gfs_filepath) as gfs_src:
        gfs_data = gfs_src.read()
        # print("reprojected", (gfs_data[0] < 20).sum(), (gfs_data[1] > 3).sum(), (gfs_data[2] > 0).sum())
        print("reprojected", (gfs_data[0] < 20).sum(), (gfs_data[1] > 0).sum())

        gfs_transform = gfs_src.transform
        gfs_width = gfs_src.width
        gfs_height = gfs_src.height
        gfs_x, gfs_y = compute_coordinates(gfs_transform, gfs_width, gfs_height)

    gfs_points = np.column_stack((gfs_x.flatten(), gfs_y.flatten()))
    interpolated_data = np.empty((gfs_data.shape[0], insat_height, insat_width))

    # Perform the interpolation for each GFS band
    print("gfs shape: ", gfs_data.shape)
    for band_idx in range(gfs_data.shape[0]):
        gfs_values = gfs_data[band_idx, :, :].flatten()

        # Interpolate the GFS data onto the INSAT grid
        interpolated_band = griddata(gfs_points, gfs_values, insat_points, method='linear')

        # Handle potential NaN values after interpolation
        interpolated_band = np.nan_to_num(interpolated_band)

        # Reshape the interpolated band to match the INSAT grid shape
        interpolated_data[band_idx, :, :] = interpolated_band.reshape(insat_x.shape)

    print("interpolated shape: ", interpolated_data.shape)
    # print("interpolated", (interpolated_data[0] < 20).sum(), (interpolated_data[1] > 0).sum(), (interpolated_data[2] > 0).sum())
    print("interpolated", (interpolated_data[0] < 20).sum(), (interpolated_data[1] > 0).sum())
    
    interpolated_gfs_data_list.append(interpolated_data)
    print(f"Interpolation complete for file {idx + 1}. Shape of interpolated data: {interpolated_data.shape}")

print(len(interpolated_gfs_data_list))

21

Opened: Reprojected_GFS/gfs.0p25.2024070918.f000.tif
reprojected 253603 260890
gfs shape:  (2, 1602, 165)
interpolated shape:  (2, 3207, 3062)
interpolated 3586451 5631450
Interpolation complete for file 1. Shape of interpolated data: (2, 3207, 3062)

Opened: Reprojected_GFS/gfs.0p25.2024071000.f000.tif
reprojected 254753 261000
gfs shape:  (2, 1602, 165)
interpolated shape:  (2, 3207, 3062)
interpolated 3764333 5780343
Interpolation complete for file 2. Shape of interpolated data: (2, 3207, 3062)

Opened: Reprojected_GFS/gfs.0p25.2024071006.f000.tif
reprojected 112561 260785
gfs shape:  (2, 1602, 165)
interpolated shape:  (2, 3207, 3062)
interpolated 3967063 5152978
Interpolation complete for file 3. Shape of interpolated data: (2, 3207, 3062)

Opened: Reprojected_GFS/gfs.0p25.2024071012.f000.tif
reprojected 253451 260674
gfs shape:  (2, 1602, 165)
interpolated shape:  (2, 3207, 3062)
interpolated 3841085 5258708
Interpolation complete for file 4. Shape of interpolated data: (2, 3

In [ ]:
import joblib

joblib.dump(interpolated_gfs_data_list, 'interpolated_gfs_data_list.joblib')

['interpolated_gfs_data_list.joblib']

In [ ]:
import numpy as np
from collections import Counter

folder = Path("GFS_RHT")

gfs_files = [file for file in folder.iterdir() if file.is_file() and file.name.startswith("gfs")]
print(f"Number of GFS files: {len(gfs_files)}")
eligibleCI_list = []

for i in range(len(gfs_files)):
    interpolated_gfs = interpolated_gfs_data_list[i]

    # print("KI: ", interpolated_gfs[0].min(), interpolated_gfs[0].max())
    # print("LI: ", interpolated_gfs[1].min(), interpolated_gfs[1].max())

    # interpolated_gfs[0] = KI, interpolated_gfs[1] = LI
    # elgibleCI = np.where((interpolated_gfs[2] > 0) & (interpolated_gfs[1] > 3) & (interpolated_gfs[0] < 20), 0,       # Stable (LI>0, SHW>3, KI<20 > 0) → 0
    #             np.where((interpolated_gfs[2] < -3) & (interpolated_gfs[1] < -3) & (interpolated_gfs[0] > 30), 2,     # Unstable (LI<-3, SHW<-3, KI>30 < -2) → 2
    #             1))                                                                                                   # all other cases → 1
    # eligibleCI_list.append(elgibleCI) 

    eligibleCI = np.where((interpolated_gfs[1] > 0) & (interpolated_gfs[0] < 20), 0,       # Stable (LI>0, KI<20) → 0
                np.where((interpolated_gfs[1] < -3) & (interpolated_gfs[0] > 30), 2,     # Unstable (LI<-3, KI>30) → 2
                1))                                                                     # all other cases → 1
    eligibleCI_list.append(eligibleCI)

    # Print the shape of eligibleCI filter of a particular file and the counts
    print(f"Eligible CI shape {Counter(eligibleCI.flatten())}")

print(len(eligibleCI_list))

# Save the eligibleCI_list in a joblib file
joblib.dump(eligibleCI_list, "eligibleCI_list.joblib")

Number of GFS files: 21
Eligible CI shape Counter({1: 4971076, 0: 3352892, 2: 1495866})
Eligible CI shape Counter({1: 4747272, 0: 3491441, 2: 1581121})
Eligible CI shape Counter({1: 4138019, 0: 3694543, 2: 1987272})
Eligible CI shape Counter({1: 4241131, 0: 3597431, 2: 1981272})
Eligible CI shape Counter({1: 4658326, 0: 3712115, 2: 1449393})
Eligible CI shape Counter({1: 4602109, 0: 3845594, 2: 1372131})
Eligible CI shape Counter({1: 4365207, 0: 3829840, 2: 1624787})
Eligible CI shape Counter({1: 4567743, 0: 3721364, 2: 1530727})
Eligible CI shape Counter({1: 4831952, 0: 3854702, 2: 1133180})
Eligible CI shape Counter({1: 4671069, 0: 3977184, 2: 1171581})
Eligible CI shape Counter({1: 4363358, 0: 4014007, 2: 1442469})
Eligible CI shape Counter({1: 4519469, 0: 3854132, 2: 1446233})
Eligible CI shape Counter({1: 4793626, 0: 3923275, 2: 1102933})
Eligible CI shape Counter({1: 4523059, 0: 4071793, 2: 1224982})
Eligible CI shape Counter({1: 4165157, 0: 4131604, 2: 1523073})
Eligible CI shap

['eligibleCI_list.joblib']